In [ ]:
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

def knearestneighbors(file, k, attributes):
    # Load and preprocess the data
    df = pd.read_csv(file)
    df = df.rename({'num': 'disease'}, axis=1)
    df['disease'] = df['disease'].apply(lambda x: min(x, 1))

    # Select features and target
    X = df[attributes].values
    y = df['disease'].values
    
    # Initialize Nearest Neighbors model
    nn = NearestNeighbors(n_neighbors=k, metric='euclidean', algorithm='auto')

    # Track precision, recall, and F1 scores for 10 iterations
    test_sizes = [0.1, 0.15, 0.2, 0.25, 0.3, 0.33, 0.4, 0.45, 0.5, 0.6]
    precision_scores = []
    recall_scores = []
    f1_scores = []
    
    for _ in range(10):  # 10 iterations
        # Randomly choose a test size and perform train-test split
        test_size = random.choice(test_sizes)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random.randint(1, 100))

        # Fit the Nearest Neighbors model on the training set
        nn.fit(X_train)

        # Store predictions for the test set
        y_pred = []
        for test_point in X_test:
            distances, indices = nn.kneighbors([test_point])
            nbrs = df.iloc[indices[0]]

            # Get the majority class among the k-nearest neighbors
            healthy = nbrs[nbrs['disease'] == 0].count().disease
            sick = nbrs[nbrs['disease'] == 1].count().disease
            predict = 0 if (healthy > sick) else 1
            y_pred.append(predict)

        # Evaluate the predictions: precision, recall, F1 score
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)


    for i, precision in enumerate(precision_scores):
        print(f"Iteration: {i + 1}")  # Start iteration count from 1 instead of 0
        print(f"Precision Score: {precision_scores[i]}")
        print(f"Recall Score: {recall_scores[i]}")
        print(f"F1 Score: {f1_scores[i]}")
        print()
    print("Mean F1 Score:", sum(f1_scores) / len(f1_scores))


## Part I

First, we needed to determine the optimal value of k for this dataset. We did this using the elbow method.

In [ ]:
def get_scores(k, df):
  # Use knn on age. First create a nearest neighbors object.
  nn = NearestNeighbors(n_neighbors=k, metric='euclidean', algorithm='auto')

  # This builds an index data structure under the hood for query performance
  X = df[['age_s', 'trestbps_s']].values
  fit = nn.fit(X)

  # Get random patients to test on
  n = 50
  patients = df.sample(n)
  patientsX = patients[['age_s','trestbps_s']].values
  patientsy = patients[['disease']].values
  # display(patients)

  # Find the k nearest neighbors to the patient.
  distances, indices = fit.kneighbors(patientsX)

  y_pred = []
  for i in range(n):
      nbrs = df.iloc[indices[i]]
      # Drop the patient of interest
      nbrs = nbrs.drop(patients.index[i], errors='ignore')

      healthy = nbrs[nbrs.disease == 0].count().disease
      sick = nbrs[nbrs.disease == 1].count().disease
      predict = 0 if (healthy > sick) else 1
      y_pred.append(predict)

  return precision_recall_fscore_support(patientsy, y_pred, labels=[1])

def determine_k(file):
  df = pd.read_csv(file)
  df['disease'] = df.num.apply(lambda x: min(x,1))
  df['age_s'] = (df.age - df.age.mean())/df.age.std()
  df['trestbps_s'] = (df.trestbps - df.trestbps.mean())/df.trestbps.std()

  kvals = range(2, 250)
  scores = [get_scores(k, df) for k in kvals]
  scores = [(p[0], r[0], f[0], s[0]) for (p,r,f,s) in scores]
  scores = list(zip(*scores))

  k_options = pd.DataFrame(
    {'f score': scores[2],
     'k': kvals
    })

  k_options = k_options.sort_values(by='f score', ascending=False)

  return (kvals, scores[2], k_options)

In [ ]:
result = determine_k('cleveland.csv')
result[2].head()

In [ ]:
plt.plot(result[0], result[1])
plt.xlabel('k')
plt.ylabel('f score')

Now, we need to determine the optimal set of attributes to use.

(TODO)

Now that we've determined the optimal k and set of attributes, we can use the k nearest neighbors method to predict whether patients have heart disease. We'll also evaluate our model.

In [ ]:
# Call the function
knearestneighbors('cleveland.csv', 40, ['age', 'trestbps', 'chol', 'thalach'])

## Part II

Now, we will repeat the same process for our second data set. This data set contains information about urls and whether they are known to be phishing or legitimate. 

Source: https://archive.ics.uci.edu/dataset/967/phiusiil+phishing+url+dataset